# Homework 6

In this homework, you're asked to write the class of **Multivariate** ordinary least square (OLS) regression with Numpy and test its performance with real-world dataset. Please fill the code block cells with your code and comments, run everything (select cell in the menu, and click Run all), save the notebook, and upload it to canvas.

In [1]:
# import the packages
import numpy as np 

## Task 1: Define the class for multivariate linear regression

Define a class named `MyLinearRegression` for the multivariate linear regression machine learning problem. It should contain a method called `fit` to estimate parameters, `predict` to generate predictions, and `score` to evaluate performance with $R^{2}$ value.

**In Task 1, you should write your code with pure Python or Numpy, and are not allowed to use any other packages/functions in Scikit-Learn**

*Hints*:

- For basic structures of this class, you can refer to the single-variable linear regression class defined in lecture notes because they are very similar. You only need to replace the formulas with the mulivariate regression case.


- Please review the mathematical part of lecture notes 10 carefully before writing the code. All the formulas used here are already given in the lecture notes, and you need to pick up the correct formulas to estimate parameters/generate predictions/evaluate performance.


- The most tricky part is about dealing with the intercepts $\beta_{0}$, while we have already done it for you in the `fit` method below. 


- For linear algebra operations in Numpy, you can consult [here](https://numpy.org/doc/stable/reference/routines.linalg.html). You can also review TA's discussion notes on Numpy.

In [2]:
class MyLinearRegression:
    '''
    Create an instance of a multivariate linear regression model given input X and y data
    '''
        
    def fit(self, X, y):
        '''
        Determine the optimal parameter B (OLS estimator) for the input data x and y   
        Parameters  x : 2D numpy array with shape (n_samples, attributes) from training data
                    y : 1D numpy array with shape (n_samples,) from training data    
        Returns an instance of self, with new attribute b containing B intercept and B coefficients 
        '''

        ones = np.ones((X.shape[0],1)) # column of ones 
        X_aug = np.concatenate((ones, X), axis = 1) # the augmented matrix, \tilde{X} in our lecture
        
        #Following formula 𝛽̂ =(𝑋̃^𝑇@𝑋̃)^−1 @ 𝑋̃^𝑇 @ 𝑌
        X_t = np.transpose(X_aug)
        a = np.matmul(X_t,X_aug)
        b = np.matmul(X_t, y_train)
        a_i = np.linalg.inv(a)
        B = np.matmul(a_i,b)
        self.b = B
            
    def predict(self,X):
        '''
        Predict the output y values for the input value X, based on trained parameter b   
        Parameters x : 2D numpy array from training or test data      
        Returns 1D numpy array (length,) based on X's shape (length, attributes), containing predicted y values
        '''
        ones = np.ones((X.shape[0],1))
        X_aug = np.concatenate((ones, X), axis = 1)
        #Following formula y_pred = X_tilde @ 𝛽̂
        y_pred = np.matmul(X_aug, self.b)  
        return y_pred
    
    def score(self, X, y):
        '''
        Calculate the R-squared based on input X and y
        Parameters x : 2D numpy array with shape (n_samples, attributes) from training or test data
                   y : 1D numpy array with shape (n_samples,) from training or test data
        Returns float, the R^2 value
        '''
        N = y.shape[0]
        MSE = (1/N) * np.sum((y - self.predict(X))**2)
        VarY = (1/N) * np.sum((y - np.mean(y))**2)
        R = 1 - MSE/VarY
        return R

## Task 2: Application to diabetes dataset

- After defining the class, we can test them with the [diabetes dataset](https://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf), loaded from scikit-learn. We're going to use 10 variables (information about each patient, already mean centered and scaled by the standard deviation) to predict the disease progression (use a number to measure, can be thought as continuous) after one year.

In [3]:
# load the dataset
from sklearn import datasets
db = datasets.load_diabetes()
print(db['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - Age
      - Sex
      - Body mass index
      - Average blood pressure
      - S1
      - S2
      - S3
      - S4
      - S5
      - S6

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).

Source URL:
https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bra

- Next, we manully split the traning and test datasets. For the basic concepts, please refer to the lecture notes/discussion files.

In [4]:
X = db['data'] # already in Numpy array format
y = db['target'] 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

- Please use the class you defined to train the linear regression model on the **training dataset**, and report the $R^{2}$ on **test dataset**.

In [5]:
reg = MyLinearRegression()
reg.fit(X_train,y_train)
R2 = reg.score(X_test,y_test)
print(R2)

0.5103954261351442


## Task 3: Comparison with Scikit-Learn

Repeat the linear regression task above (i.e. train the linear regression on the **training dataset**, and report the R^{2} on **test dataset**.) with calling the methods in [sklearn package](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Ideally, the results should be the same with task 2.

In [6]:
from sklearn import linear_model
myRegression = linear_model.LinearRegression()
myRegression.fit(X_train, y_train)
y_pred = myRegression.predict(X_train)
R2 = myRegression.score(X_test,y_test)
print(R2)

0.510395426135144


## Optional Task

1. Create a pandas dataframe of the data and use seaborn to visualize.


2. Can you try the regression module in PyCaret to do the automatic machine learning for this data? You can follow the tutorial [here](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)

In [7]:
import pandas as pd
data = pd.DataFrame(np.c_[db['data'], db['target']],columns= db['feature_names']+['target'])
data

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0
5,-0.092695,-0.044642,-0.040696,-0.019442,-0.068991,-0.079288,0.041277,-0.076395,-0.041180,-0.096346,97.0
6,-0.045472,0.050680,-0.047163,-0.015999,-0.040096,-0.024800,0.000779,-0.039493,-0.062913,-0.038357,138.0
7,0.063504,0.050680,-0.001895,0.066630,0.090620,0.108914,0.022869,0.017703,-0.035817,0.003064,63.0
8,0.041708,0.050680,0.061696,-0.040099,-0.013953,0.006202,-0.028674,-0.002592,-0.014956,0.011349,110.0
9,-0.070900,-0.044642,0.039062,-0.033214,-0.012577,-0.034508,-0.024993,-0.002592,0.067736,-0.013504,310.0


In [8]:
# write your codes here